In [1]:
import os
import argparse
import baseline
import sys
from EigenTrajectory import *
from utils import *
# Check if running in Jupyter environment
if 'ipykernel' in sys.argv[0]:
    sys.argv = [sys.argv[0]]

parser = argparse.ArgumentParser()
parser.add_argument('--cfg', default="./config/eigentrajectory-stgcnn-hotel.json", type=str, help="config file path")
parser.add_argument('--tag', default="EigenTrajectory-TEMP", type=str, help="personal tag for the model")
# parser.add_argument('--gpu_id', default="0", type=str, help="gpu id for the model")
parser.add_argument('--test', default=False, action='store_true', help="evaluation mode")
args = parser.parse_args()

print("===== Configs =====")
hyper_params = get_exp_config(args.cfg)
print_arguments(hyper_params)
PredictorModel = getattr(baseline, hyper_params.baseline).TrajectoryPredictor
hook_func = DotDict({"model_forward_pre_hook": getattr(baseline, hyper_params.baseline).model_forward_pre_hook,
                        "model_forward": getattr(baseline, hyper_params.baseline).model_forward,
                        "model_forward_post_hook": getattr(baseline, hyper_params.baseline).model_forward_post_hook})
ModelTrainer = getattr(trainer, *[s for s in trainer.__dict__.keys() if hyper_params.baseline in s.lower()])

===== Configs =====
dataset_dir: ./datasets/ | checkpoint_dir: ./checkpoints/ | dataset: hotel | traj_dim: 2 | 
obs_len: 8 | obs_step: 10 | pred_len: 12 | pred_step: 10 | skip: 1 | k: 6 | static_dist: 0.353 | 
num_samples: 20 | obs_svd: True | pred_svd: True | baseline: stgcnn | batch_size: 32 | 
num_epochs: 128 | lr: 0.001 | weight_decay: 0.0001 | clip_grad: 10 | lr_schd: True | 
lr_schd_step: 64 | lr_schd_gamma: 0.5


In [2]:
trainer = ModelTrainer(base_model=PredictorModel, model=EigenTrajectory, hook_func=hook_func,
                        args=args, hyper_params=hyper_params)
# print(PredictorModel.in_channels)
if not args.test:
    trainer.init_descriptor()
    trainer.fit()
else:
    trainer.load_model()
    print("Testing...", end=' ')
    results = trainer.test()
    print(f"Scene: {hyper_params.dataset}", *[f"{meter}: {value:.8f}" for meter, value in results.items()])

Trainer initiating...
Checkpoint dir: ./checkpoints//EigenTrajectory-TEMP/hotel/


c:\Users\moham\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\moham\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\moham/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:31<00:00, 1.47MB/s]


ET descriptor initialization...
Anchor generation...
Training started...


Train Epoch 0:   1%|          | 9/1669 [01:02<3:11:03,  6.91s/it]